In [78]:
#!pip3 install folium
#!pip3 install wget
import folium
import wget
import pandas as pd
# Import folium MarkerCluster plugin
#from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
#from folium.plugins import MousePosition
# Import folium DivIcon plugin
#from folium.features import DivIcon
spacex_csv_file = wget.download('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv')
spacex_df=pd.read_csv(spacex_csv_file)
spacex_df.head()

,Flight Number,Date,Time (UTC),Booster Version,Launch Site,Payload,Payload Mass (kg),Orbit,Customer,Mission Outcome,Landing Outcome,class,Lat,Long
0,1,2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0.0,LEO,SpaceX,Success,Failure (parachute),0,28.562302,-80.577356
1,2,2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel o...",0.0,LEO (ISS),NASA (COTS) NRO,Success,Failure (parachute),0,28.562302,-80.577356
2,3,2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2+,525.0,LEO (ISS),NASA (COTS),Success,No attempt,0,28.562302,-80.577356
3,4,2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500.0,LEO (ISS),NASA (CRS),Success,No attempt,0,28.562302,-80.577356
4,5,2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677.0,LEO (ISS),NASA (CRS),Success,No attempt,0,28.562302,-80.577356


In [79]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610746


In [80]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

In [81]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

In [82]:
# Initial the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
incidents = folium.map.FeatureGroup()

# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label
#launch_sites_df[['Launch Site', 'Lat', 'Long']]
for site, lat, long in zip(launch_sites_df['Launch Site'],launch_sites_df.Lat, launch_sites_df.Long):
    
    launchSite_circle = folium.Circle([lat, long], radius = 1000, color='#d35400',fill=True).add_child(folium.Popup(site))
    launchSite_marker = folium.map.Marker([lat,long], icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), 
                                              html='<div style="font-size=12; color=#d35400;"><b>%s</b></div>' % site, ))
    incidents.add_child(launchSite_circle)
    incidents.add_child(launchSite_marker)
    
site_map.add_child(incidents)
    

In [83]:
spacex_df.tail(10)


,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


In [84]:
marker_cluster = MarkerCluster()
spacex_df.head()

,Launch Site,Lat,Long,class
0,CCAFS LC-40,28.562302,-80.577356,0
1,CCAFS LC-40,28.562302,-80.577356,0
2,CCAFS LC-40,28.562302,-80.577356,0
3,CCAFS LC-40,28.562302,-80.577356,0
4,CCAFS LC-40,28.562302,-80.577356,0


In [85]:
# Function to assign color to launch outcome
def assign_marker_color(launch_outcome):
    if launch_outcome == 1:
        return 'green'
    else:
        return 'red'
    
spacex_df['marker_color'] = spacex_df['class'].apply(assign_marker_color)
spacex_df.tail(10)

,Launch Site,Lat,Long,class,marker_color
46,KSC LC-39A,28.573255,-80.646895,1,green
47,KSC LC-39A,28.573255,-80.646895,1,green
48,KSC LC-39A,28.573255,-80.646895,1,green
49,CCAFS SLC-40,28.563197,-80.576820,1,green
50,CCAFS SLC-40,28.563197,-80.576820,1,green
51,CCAFS SLC-40,28.563197,-80.576820,0,red
52,CCAFS SLC-40,28.563197,-80.576820,0,red
53,CCAFS SLC-40,28.563197,-80.576820,0,red
54,CCAFS SLC-40,28.563197,-80.576820,1,green
55,CCAFS SLC-40,28.563197,-80.576820,0,red


In [86]:
# Add marker_cluster to current site_map
site_map.add_child(marker_cluster)

# for each row in spacex_df data frame
# create a Marker object with its coordinate
# and customize the Marker's icon property to indicate if this launch was successed or failed, 
# e.g., icon=folium.Icon(color='white', icon_color=row['marker_color']
for index, record in spacex_df.iterrows():
    # TODO: Create and add a Marker cluster to the site map
    icon_=folium.Icon(color='white', icon_color=record['marker_color'])
    marker = folium.Marker([record['Lat'],record['Long']], icon=icon_)
    marker_cluster.add_child(marker)

site_map

In [87]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

In [88]:
# find coordinate of the closet coastline
# e.g.,: Lat: 28.56367  Lon: -80.57163
#distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)
distance_coastline = calculate_distance(28.563197,-80.576820, 28.56399, -80.56811)

In [89]:
distance_marker = folium.Marker([28.56399, -80.56811], icon=DivIcon(icon_size=(20,20), icon_anchor=(0,0), 
                                            html='<div style="font-size=12; color="#d35400";> <b>%s</b></div>' % "{:10.2f} KM".format(distance_coastline),))
                                             
site_map.add_child(distance_marker) 

In [90]:
coordinates = [[28.563197,-80.576820],[28.56399, -80.56811]]
lines=folium.PolyLine(locations=coordinates, weight=1)
site_map.add_child(lines)

In [91]:
#Nearest hwy :- Samuel C Philips Pkwy
#hwy_coordinates=[28.56253,-80.57066]
#Nearest railway :- Nasa Railway coordinates
nasa_railway_coordinates = [28.57395, -80.58517]
#nearest city :- Cape Canaveral
cape_canaveral_city = [28.40112,-80.60719]
# Create a marker with distance to a closest city, railway, highway, etc.
# Draw a line between the marker to the launch site

city_coordinates = [28.40112,-80.60719]
railway_coordinates = [28.57395, -80.58517]
highway_coordinates = [28.56235,-80.57064]


In [92]:
coordinates = [[28.563197,-80.576820],city_coordinates]
distance_city = calculate_distance(28.563197,-80.576820, city_coordinates[0], city_coordinates[1])
distance_marker = folium.Marker(city_coordinates, icon=DivIcon(icon_size=(20,20), icon_anchor=(0,0), 
                  html='<div style="font-size=12; color="#d35400";> <b>%s</b></div>' % "{:10.2f} KM".format(distance_city),))
                                             
site_map.add_child(distance_marker) 
lines=folium.PolyLine(locations=coordinates, weight=1)
site_map.add_child(lines)

In [93]:
coordinates = [[28.563197,-80.576820],railway_coordinates]
distance_railway = calculate_distance(28.563197,-80.576820, railway_coordinates[0], railway_coordinates[1])
distance_marker = folium.Marker(railway_coordinates, icon=DivIcon(icon_size=(20,20), icon_anchor=(0,0), 
                  html='<div style="font-size=12; color="#d35400";> <b>%s</b></div>' % "{:10.2f} KM".format(distance_railway),))
                                             
site_map.add_child(distance_marker) 
lines=folium.PolyLine(locations=coordinates, weight=1)
site_map.add_child(lines)

In [94]:
coordinates = [[28.563197,-80.576820],highway_coordinates]
distance_hwy = calculate_distance(28.563197,-80.576820, highway_coordinates[0], highway_coordinates[1])
distance_marker = folium.Marker(highway_coordinates, icon=DivIcon(icon_size=(20,20), icon_anchor=(0,0), 
                  html='<div style="font-size=12; color="#d35400";> <b>%s</b></div>' % "{:10.2f} KM".format(distance_hwy),))
                                             
site_map.add_child(distance_marker) 
lines=folium.PolyLine(locations=coordinates, weight=1)
site_map.add_child(lines)